In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
import random
import torch

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
y = train['diagnosis']
x = train.drop('diagnosis', axis=1)

In [5]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x = sc.fit_transform(x)
test = sc.transform(test)

In [6]:
x_tensor = torch.FloatTensor(x)
y_tensor = torch.FloatTensor(y).reshape(-1, 1)
test_tensor = torch.FloatTensor(test)

In [7]:
layer1 = torch.nn.Linear(x_tensor.shape[1], 64)
layer2 = torch.nn.Linear(64, 1)

ReLU = torch.nn.ReLU()
Sigmoid = torch.nn.Sigmoid()

model = torch.nn.Sequential(layer1, ReLU, layer2, Sigmoid)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

loss = torch.nn.BCELoss()

In [8]:
epochs = 10000

for epoch in range(epochs):
    optimizer.zero_grad()
    hypothesis = model(x_tensor)
    cost = loss(hypothesis, y_tensor)
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'{epoch}/{epochs} cost: {cost.item()}')

0/10000 cost: 0.6697134375572205
100/10000 cost: 0.37863317131996155
200/10000 cost: 0.2435004860162735
300/10000 cost: 0.17958880960941315
400/10000 cost: 0.1448194980621338
500/10000 cost: 0.12317901104688644
600/10000 cost: 0.10829102993011475
700/10000 cost: 0.0973493754863739
800/10000 cost: 0.08885503560304642
900/10000 cost: 0.08202193677425385
1000/10000 cost: 0.07642055302858353
1100/10000 cost: 0.07173861563205719
1200/10000 cost: 0.06774929910898209
1300/10000 cost: 0.06427604705095291
1400/10000 cost: 0.0612231008708477
1500/10000 cost: 0.0585181787610054
1600/10000 cost: 0.056063394993543625
1700/10000 cost: 0.05383564904332161
1800/10000 cost: 0.05182093381881714
1900/10000 cost: 0.05000794306397438
2000/10000 cost: 0.04835604131221771
2100/10000 cost: 0.04683802276849747
2200/10000 cost: 0.04544239863753319
2300/10000 cost: 0.044151052832603455
2400/10000 cost: 0.042956236749887466
2500/10000 cost: 0.041848115622997284
2600/10000 cost: 0.04081657901406288
2700/10000 cost

In [10]:
predict = [int(x >= 0.5) for x in model(test_tensor)]
submit = pd.read_csv('submit_sample.csv')
submit['diagnosis'] = predict
submit.to_csv('submit.csv', index=False)